In [1]:
import numpy as np
import tensorflow as tf

/home/jupyter/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyter/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyter/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyter/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
# import mnist data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/home/jupyter/.data/mnist", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/jupyter/.data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/jupyter/.data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /home/jupyter/.data/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/jupyter/.data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# define parameters
element_size = 28
time_steps = 28
num_classes = 10
batch_size = 128
hidden_layer_size = 128

In [1]:
LOG_DIR = "./logs/RNN_with_summaries"

In [5]:
_inputs = tf.placeholder(tf.float32, shape=
                          [None, time_steps, element_size],
                         name='inputs')
y = tf.placeholder(tf.float32, shape=
                  [None, num_classes], name='labels')

In [6]:
batch_x, batch_y = mnist.train.next_batch(batch_size)
batch_x = batch_x.reshape((batch_size, time_steps, element_size))

In [7]:
# this helper function, taken from the official TensorFlow docs,
# simply adds some ops that take care of logging summaries
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [8]:
# weighs and bias for input and hidden layer
with tf.name_scope('rnn_weights'):
    with tf.name_scope('W_x'):
        Wx = tf.Variable(tf.zeros([element_size, hidden_layer_size]))
        variable_summaries(Wx)
    with tf.name_scope('W_h'):
        Wh = tf.Variable(tf.zeros([hidden_layer_size, hidden_layer_size]))
        variable_summaries(Wh)
    with tf.name_scope('Bias'):
        b_rnn = tf.Variable(tf.zeros([hidden_layer_size]))
        variable_summaries(b_rnn)

In [9]:
def rnn_step(previous_hidden_state, x):
    current_hidden_state = tf.tanh(
        tf.matmul(previous_hidden_state, Wh) +
        tf.matmul(x, Wx) + b_rnn
    )
    return current_hidden_state

* processing inputs to work with scan function
* current input shape: (batch_size, time_steps, element_size)

In [11]:
processed_input = tf.transpose(_inputs, perm=[1, 0, 2])

* current input shape now: (time_steps, batch_size, element_size)


In [12]:
initial_hidden = tf.zeros([batch_size, hidden_layer_size])

* getting all state vectors across time
* important to the first axis represents the time axis
* we can iterate across all time steps by using the tf.scan()
* it is applies a callable (function) to a sequence of elements in order
* as explained in the folowing note

In [13]:
all_hidden_states = tf.scan(rnn_step,
                           processed_input,
                           initializer=initial_hidden,
                           name='states')

##### tf.scan()
* This important function was added to TensorFlow to allow us to
introduce loops into the computation graph, instead of just
“unrolling” the loops explicitly by adding more and more replications of the same operations. More technically, it is a higherorder
function very similar to the reduce operator, but it returns all intermediate accumulator values over time. There are several advantages to this approach, chief among them the ability to have a
dynamic number of iterations rather than fixed, computational
speedups and optimizations for graph construction.

* In this case, we use tf.scan() to sequentially concatenate characters to a string, in a
manner analogous to the arithmetic cumulative sum.

In [14]:
elems = np.array(['T', 'e', 'n', 's', 'o', 'r', ' ', 'F', 'l', 'o', 'w'])
scan_sum = tf.scan(lambda a, x: a + x, elems)

sess = tf.InteractiveSession()
sess.run(scan_sum)

array([b'T', b'Te', b'Ten', b'Tens', b'Tenso', b'Tensor', b'Tensor ',
       b'Tensor F', b'Tensor Fl', b'Tensor Flo', b'Tensor Flow'],
      dtype=object)

In [15]:
# weights for output layers
with tf.name_scope('linear_layer_weights') as scope:
    with tf.name_scope('W_linear'):
        Wl = tf.Variable(tf.truncated_normal([hidden_layer_size,
                                             num_classes],
                                            mean=0, stddev=.01))
        variable_summaries(Wl)
    with tf.name_scope('Bias_linear'):
        bl = tf.Variable(tf.truncated_normal([num_classes],
                                            mean=0, stddev=.01))
        variable_summaries(bl)

In [16]:
# apply linear layer to state vector
def get_linear_layer(hidden_state):
    return tf.matmul(hidden_state, Wl) + bl

In [17]:
with tf.name_scope('linear_layer_weights') as scope:
    # iterate across time, apply linear layer to all RNN outputs
    all_outputs = tf.map_fn(get_linear_layer, all_hidden_states)
    # get last output
    output = all_outputs[-1]
    tf.summary.histogram('outputs', output)

### RNN classification

In [18]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y)
    )
    tf.summary.scalar('cross_entropy', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)
    
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(
    tf.argmax(y, 1), tf.argmax(output, 1))
    accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32))) * 100
    tf.summary.scalar('accuracy', accuracy)

#merge all the summaries
merged = tf.summary.merge_all()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [21]:
# get a small test set
test_data = mnist.test.images[:batch_size].reshape((-1, time_steps,
                                                   element_size))
test_label = mnist.test.labels[:batch_size]

with tf.Session() as sess:
    # Write summaries to LOG_DIR -- used by TensorBoard
    train_writer = tf.summary.FileWriter(LOG_DIR + '/train',
                                graph=tf.get_default_graph())
    test_writer = tf.summary.FileWriter(LOG_DIR + '/test',
                                graph=tf.get_default_graph())
    sess.run(tf.global_variables_initializer())

    for i in range(10000):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # reshape data to get 28 sequences of 28 pixels
        batch_x = batch_x.reshape((batch_size, time_steps, element_size))
        summary,_ = sess.run([merged, train_step],
                            feed_dict={_inputs:batch_x, y:batch_y})
        # add to summaries
        train_writer.add_summary(summary, i)

        if i % 1000 == 0:
            acc,loss, = sess.run([accuracy,cross_entropy],
                                feed_dict={_inputs: batch_x,
                                            y: batch_y})
            print("Iter " + str(i) + ", Minibatch Loss= " + \
                "{:.6f}".format(loss) + ", Training Accuracy= " + \
                "{:.5f}".format(acc))
        if i % 10:
            # Calculate accuracy for 128 MNIST test images and
            # add to summaries
            summary, acc = sess.run([merged, accuracy],
                                   feed_dict={_inputs: test_data,
                                             y: test_label})
            test_writer.add_summary(summary, i)

    test_acc = sess.run(accuracy, feed_dict={_inputs: test_data,
                                                y: test_label})
    print("Test Accuracy: {}, step: {}".format(test_acc, i))

Iter 0, Minibatch Loss= 2.302763, Training Accuracy= 12.50000
Iter 1000, Minibatch Loss= 1.436974, Training Accuracy= 50.78125
Iter 2000, Minibatch Loss= 0.632700, Training Accuracy= 80.46875
Iter 3000, Minibatch Loss= 0.206060, Training Accuracy= 93.75000
Iter 4000, Minibatch Loss= 0.264060, Training Accuracy= 93.75000
Iter 5000, Minibatch Loss= 0.093984, Training Accuracy= 98.43750
Iter 6000, Minibatch Loss= 0.077726, Training Accuracy= 99.21875
Iter 7000, Minibatch Loss= 0.059325, Training Accuracy= 98.43750
Iter 8000, Minibatch Loss= 0.022851, Training Accuracy= 99.21875
Iter 9000, Minibatch Loss= 0.009136, Training Accuracy= 100.00000
Test Accuracy: 97.65625, step: 9999
